In [1]:
import cv2
import biaoding1
import numpy as np
from Stitcher import Stitcher
import glob
import os

### 去畸变参数读取

In [2]:
DIM = []
K = []
D = []
CAMERA_COUNT = 3  # 摄像头个数

for index in range(1, CAMERA_COUNT + 1):
    DIM.append(np.load(r"D:/大四上/传感器/全景拼接/data/DIM" + str(index) + ".npy"))
    K.append(np.load(r"D:/大四上/传感器/全景拼接/data/K" + str(index) + ".npy"))
    D.append(np.load(r"D:/大四上/传感器/全景拼接/data/D" + str(index) + ".npy"))

### 函数定义

In [45]:
#SIFT算法（返回特征点集，及对应的特征向量集）
def detectAndDescribe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 建立SIFT生成器
    descriptor = cv2.xfeatures2d.SIFT_create()
    # 检测SIFT特征点，并计算描述子
    (kps, features) = descriptor.detectAndCompute(image, None)

    # 将结果转换成NumPy数组
    kps = np.float32([kp.pt for kp in kps])

    # 返回特征点集，及对应的描述特征
    return (kps, features)

#Brute-Force蛮力匹配算法
def matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
    # 建立暴力匹配器
    matcher = cv2.BFMatcher()

    # 使用KNN检测来自A、B图的SIFT特征匹配对，K=2
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)

    matches = []
    for m in rawMatches:
        # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        # 存储两个点在featuresA, featuresB中的索引值
            matches.append((m[0].trainIdx, m[0].queryIdx))

    # 当筛选后的匹配对大于4时，计算视角变换矩阵
    if len(matches) > 4:
        # 获取匹配对的点坐标
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])

        # 计算视角变换矩阵
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

        # 返回结果
        return (matches, H, status)

    # 如果匹配对小于4时，返回None
    return None

def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    # 初始化可视化图片，将A、B图左右连接到一起
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # 联合遍历，画出匹配对
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        # 当点对匹配成功时，画到可视化图上
        if s == 1:
            # 画出匹配对
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

    # 返回可视化结果
    return vis

def stitch(images, ratio=0.9, reprojThresh=4.0,showMatches=False):
    #获取输入图片
    (imageB, imageA) = images
    #检测A、B图片的SIFT关键特征点，并计算特征描述子
    (kpsA, featuresA) = detectAndDescribe(imageA)
    (kpsB, featuresB) = detectAndDescribe(imageB)

    # 匹配两张图片的所有特征点，返回匹配结果
    M = matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)

    # 如果返回结果为空，没有匹配成功的特征点，退出算法
    if M is None:
        return None

    # 否则，提取匹配结果
    # H是3x3视角变换矩阵
    (matches, H, status) = M
    # 将图片A进行视角变换，result是变换后图片
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
    #cv_show('result', result)
    # 将图片B传入result图片最左端
    result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
    #cv_show('result', result)
    # 检测是否需要显示图片匹配
    if showMatches:
        # 生成匹配图片
        vis = drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
        # 返回结果
        return (result,H,vis)

    # 返回匹配结果
    return result

In [25]:
def offlinestitch(images,H):
    (imageB, imageA) = images
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
    result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
    return result

### 载入摄像头

In [47]:
s1 = cv2.VideoCapture(0)
s1.set(3,1920)
s1.set(4,1080)
s2 = cv2.VideoCapture(1)
s2.set(3,1920)
s2.set(4,1080)
s3 = cv2.VideoCapture(2)
s3.set(3,1920)
s3.set(4,1080)

True

### 主程序

In [48]:
# stitcher=Stitcher()
index=0
# sum_H=np.zeros((3,3))
# H_2_3=np.zeros((3,3))
# H_index=[]
# saveLocation=r'D:/imageStitching/1212/stitch_2_3/'
# if not os.path.exists(saveLocation):
#     os.makedirs(saveLocation)
if s1.isOpened():
# if s2.isOpened():
#     if s3.isOpened():
    while True:
        ret1,frame1=s1.read()
        frame1=frame1[0:1080,390:1470]
        frame1=biaoding1.undistort_frames(frame1,DIM[0], K[0], D[0])
        # frame1=cv2.resize(frame1,(360,360))

        ret2,frame2=s2.read()
        frame2=frame2[0:1080,390:1470]
        frame2 = biaoding1.undistort_frames(frame2, DIM[1], K[1], D[1])
        #             frame2 = cv2.resize(frame2, (360, 360))

        ret3,frame3=s3.read()
        frame3=frame3[0:1080,390:1470]
        frame3 = biaoding1.undistort_frames(frame3, DIM[2], K[2], D[2])
        # #             frame3 = cv2.resize(frame3, (360, 360))

        (result_right,H,vis_right) = stitch([frame2, frame3], showMatches=True)
        result_right=cv2.resize(result_right,None,fx=0.5, fy=0.5)
        # (result_left, vis_left) = stitcher.stitchleft([frame1, result_right], showMatches=True)
        #             H_index.append(H)
        #             sum_H=H+sum_H
        #             H_2_3=sum_H/index
        #             print(H_2_3)
#         cv2.imshow("camera1", frame1)
        cv2.imshow("camera2", frame2)
        cv2.imshow("camera3", frame3)
        cv2.imshow("result_right", result_right)
        #             cv2.imshow("vis", vis_right)
        #             cv2.imwrite(saveLocation+str(index)+'.jpg', result_right)
        # cv2.imshow("result",result_left)
        #             index=index+1
        #             if index>=100:
        #                 break
        if cv2.waitKey(25) & 0xff==(ord('q')):
            break

s1.release()
s2.release()
s3.release()
cv2.destroyAllWindows()

### 用离线参数拼接

In [67]:
np.save(r"D:/大四上/传感器/全景拼接/data/H_2_3.npy",H_2_3)

In [69]:
H_2_3=[]
H_2_3=np.load(r"D:/大四上/传感器/全景拼接/data/H_2_3.npy",H_2_3)
if s2.isOpened():
    if s3.isOpened():
        while True:
            index=index+1
            ret2,frame2=s2.read()
            frame2 = biaoding1.undistort_frames(frame2, DIM[1], K[1], D[1])
            ret3,frame3=s3.read()
            frame3 = biaoding1.undistort_frames(frame3, DIM[2], K[2], D[2])
            result=offlinestitch([frame2, frame3],H_2_3)
            cv2.imshow("result", result)
            if cv2.waitKey(25) & 0xff==(ord('q')):
                break
s1.release()
s2.release()
s3.release()
cv2.destroyAllWindows()


### 获取离线图片

In [25]:
index=0
saveLocation=r'D:/imageStitching/1212/stitch_2_3/'
if not os.path.exists(saveLocation):
    os.makedirs(saveLocation)
if s1.isOpened():
    if s2.isOpened():
        if s3.isOpened():
            while True:
#                 ret1,frame1=s1.read()
#                 frame1=biaoding1.undistort_frames(frame1, DIM[0], K[0], D[0])
#                 # frame1=cv2.resize(frame1,(360,360))

                ret2,frame2=s2.read()
                frame2 = biaoding1.undistort_frames(frame2, DIM[1], K[1], D[1])
                # frame2 = cv2.resize(frame2, (360, 360))

                ret3,frame3=s3.read()
                frame3 = biaoding1.undistort_frames(frame3, DIM[2], K[2], D[2])
    #                 frame3 = cv2.resize(frame3, (360, 360))

#                 cv2.imshow("camera1", frame1)
                cv2.imshow("camera2", frame2)
                cv2.imshow("camera3", frame3)
                index=index+1
                
                if index<=20:
                    continue
#                 cv2.imwrite(saveLocation+r"camera2_"+str(index)+'.jpg', frame1)
                cv2.imwrite(saveLocation+r"camera2_"+str(index)+'.jpg', frame2)
                cv2.imwrite(saveLocation+r"camera3_"+str(index)+'.jpg', frame3)
                if index>=100:
                    break
                if cv2.waitKey(25) & 0xff==(ord('q')):
                    break

s1.release()
s2.release()
s3.release()
cv2.destroyAllWindows()
    

### 拼接静态图片

In [32]:
saveLocation=r'D:/imageStitching/1212/stitch_2_3/'

index=100
# 读取拼接图片
imageA = cv2.imread(saveLocation+r"camera2_"+str(index)+'.jpg')
imageB = cv2.imread(saveLocation+r"camera3_"+str(index)+'.jpg')

# 把图片拼接成全景图
stitcher = Stitcher() #创建类的实例化
(result, vis) = stitcher.stitchleft([imageA, imageB], showMatches=True) #调用类中的函数

# 显示所有图片
stitcher.cv_show("Keypoint Matches", vis)
stitcher.cv_show("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
for index in range(1, 4):
    DIM.append(np.load(r"D:/大四上/传感器/全景拼接/data/DIM" + str(index) + ".npy"))
    K.append(np.load(r"D:/大四上/传感器/全景拼接/data/K" + str(index) + ".npy"))
    D.append(np.load(r"D:/大四上/传感器/全景拼接/data/D" + str(index) + ".npy"))
    print(K[index-1])

[[160.81101317   0.         242.33570829]
 [  0.         161.04777673 236.6130631 ]
 [  0.           0.           1.        ]]
[[159.60370542   0.         225.59037695]
 [  0.         159.9038378  232.02681161]
 [  0.           0.           1.        ]]
[[163.76812015   0.         239.09298896]
 [  0.         163.97747475 236.92056073]
 [  0.           0.           1.        ]]


### 12.13demo

In [34]:
if s1.isOpened():
    while True:
        ret,frame=s1.read()
        frame1=biaoding1.undistort_frames(frame, DIM[1], K[1], D[1])
        cv2.imshow("camera1", frame1)
        if cv2.waitKey(25) & 0xff==(ord('q')):
            break
s1.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

### 图片集去畸变

In [3]:
Images=[]
for index in range(1,4):
    Images.append(sorted(glob.glob(r"D:/imageStitching/waicanbiaoding1/camera"+str(index)+"/*.jpg"),key=os.path.getmtime))
    
saveLocation=[]
for index in range(1,4):
    saveLocation.append(r"D:/imageStitching/waicanbiaoding1/camera"+str(index)+"_undistorted/")
    
for index in range(3):
    for fname in Images[index]:
        undistorted_img=biaoding1.undistort_Images(fname,DIM[index],K[index],D[index],saveLocation[index])    

    